In [1]:
import subprocess

from itertools import groupby
from bs4 import BeautifulSoup

In [2]:
def request(url, **kwargs):
    command = f'wget -qO- {url}'
    
    if user := kwargs.get('user'):
        command += f' --user {user}'
    
    if password := kwargs.get('password'):
        command += f' --password {password}'

    output = subprocess.run(command, shell=True, capture_output=True)
    
    return output.stdout

In [3]:
def partition_directories(hrefs):
    directories = []
    files = []

    for is_directory, group in groupby(hrefs, lambda link: link.endswith('/')):
        for item in group:
            if is_directory:
                if item != '../':
                    directories.append(item)
            else:
                files.append(item)
                
    return (directories, files)

In [4]:
def get_links(url, **kwargs):
    root = request(url, **kwargs)
    soup = BeautifulSoup(root)
    hrefs = [a.get('href') for a in soup.find_all('a', href=True)]
    directories, files = partition_directories(hrefs)
    
    links = [url + file for file in files]
    
    for directory in directories:
        root_url = url + directory
        links += get_links(root_url, **kwargs)
        
    return links

In [5]:
def create_aria2_input_file(links, output, **kwargs):
    with open(output, 'w') as f:
        for link in links:
            download_link = link + '?download'
            lines = [download_link]
            
            if header := kwargs.get('header'):
                lines.append(f' header={header}')

            file_path = link.replace(root_url, '')
            
            output_dir = kwargs.get('dir', '')
            

            if '/' in file_path:
                file = file_path.split('/')[-1]
                output_dir += file_path.replace(file, '')
            
            if output_dir:
                lines.append(f' dir={output_dir}')

            f.writelines(line + '\n' for line in lines)

In [6]:
user = 'hphuang'
password = 'azf0vxe.zpj.CYG1pqc'

In [7]:
root_url = 'https://physionet.org/files/nch-sleep/3.1.0/'

In [8]:
links = get_links(root_url, user=user, password=password)

In [9]:
create_aria2_input_file(links, 'output.txt', dir='/var/share/aria2c/', header='Cookie: csrftoken=dAOa5FC7UydRAelbxP2vmK77DlX5ywsZET8Ci6QXrBpIQdJy3bT4SROyqSDMPTQZ; sessionid=4tklree31b5ll76ejd9t8ve0z9g260q5')